In [2]:
from pprint import pprint

# itertools

## What are itertools?

[itertools](https://docs.python.org/3.7/library/itertools.html) is a python module included in standard python. The documentation describes them as:

> The module standardizes a core set of fast, memory efficient tools that are useful by themselves or in combination. Together, they form an “iterator algebra” making it possible to construct specialized tools succinctly and efficiently in pure Python.

But essentially it's a module that contains a number of functions that help with iteration!

## Why bother with itertools?

[zen of python](https://en.wikipedia.org/wiki/Zen_of_Python#:~:text=The%20Zen%20of%20Python%20is,Python%20mailing%20list%20in%201999.)

>Flat is better than nested.

>Readability counts.

### Note

I use `[*generator]` a lot here, it just unpacks the whole generator into a list, so we can see everything there, it's equivalent to `[x for x in generator]` (but faster)!



In [3]:
%%timeit
[*range(100)]

759 ns ± 0.429 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
%%timeit
[x for x in range(100)]

2.49 µs ± 1.82 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Example usage

The below example has been taken from a real world unittest.

In [5]:
# Adapted from a real example
obj = [[{"a":1, "b":2, "c":3}], [{"d":4, "e":5, "f":6}], [{"g":7, "h":8, "i":9}]]
expected_obj = obj

# Triple nested loop!
for i, arglists in enumerate(obj):
    for j, arglist in enumerate(arglists):
        for key, value in expected_obj[i][j].items():
            assert value == arglist[key]

In [6]:
import itertools

zipped_values = itertools.zip_longest(
    itertools.chain.from_iterable(obj),
    itertools.chain.from_iterable(expected_obj),
)
for test_value, expected_value in zipped_values:
    assert expected_value.items() <= test_value.items()

### Which approach is faster?

In [7]:
def func1():
    for i, arglists in enumerate(obj):
        for j, arglist in enumerate(arglists):
            for key, value in expected_obj[i][j].items():
                assert value == arglist[key]
                
def func2():
    zipped_values = itertools.zip_longest(
        itertools.chain.from_iterable(obj),
        itertools.chain.from_iterable(expected_obj),
    )
    for test_value, expected_value in zipped_values:
        assert expected_value.items() <= test_value.items()

In [8]:
%%timeit

func1()

1.84 µs ± 26.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [9]:
%%timeit

func2()

1.56 µs ± 1.81 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Breaking it down

In [10]:
obj # is a 2d array with each element containing a dict

[[{'a': 1, 'b': 2, 'c': 3}],
 [{'d': 4, 'e': 5, 'f': 6}],
 [{'g': 7, 'h': 8, 'i': 9}]]

If we know the format our data structure takes, e.g. we know that obj is a 2D array, we can use `chain.from_iterable` to remove a level.

In [15]:
print(itertools.chain.from_iterable(obj)) # generator expression

# Note: we can unpack the iterable and just use chain
[*itertools.chain.from_iterable(obj)]# == [*itertools.chain(*obj)] 

[{'a': 1, 'b': 2, 'c': 3}, {'d': 4, 'e': 5, 'f': 6}, {'g': 7, 'h': 8, 'i': 9}]

In [8]:
zipped_values = itertools.zip_longest(
    itertools.chain.from_iterable(obj),
    itertools.chain.from_iterable(expected_obj),
)

[*zipped_values]

[({'a': 1, 'b': 2, 'c': 3}, {'a': 1, 'b': 2, 'c': 3}),
 ({'d': 4, 'e': 5, 'f': 6}, {'d': 4, 'e': 5, 'f': 6}),
 ({'g': 7, 'h': 8, 'i': 9}, {'g': 7, 'h': 8, 'i': 9})]

Why not just use zip?

For this toy example, there'd be no difference:

In [20]:
zipped_values = zip(
    itertools.chain.from_iterable(obj),
    itertools.chain.from_iterable(expected_obj),
)

[*zipped_values]

[({'a': 1, 'b': 2, 'c': 3}, {'a': 1, 'b': 2, 'c': 3}),
 ({'d': 4, 'e': 5, 'f': 6}, {'d': 4, 'e': 5, 'f': 6}),
 ({'g': 7, 'h': 8, 'i': 9}, {'g': 7, 'h': 8, 'i': 9})]

But in a real example, our test output may be a different length to the expected output:

In [10]:
test_result = [1, 2, 3]
expected_result = [1, 2, 3, 4, 5]

print(f"Test results match the expected: {all([i == j for i,j in zip(test_result, expected_result)])}")

[*zip(test_result, expected_result)]

Test results match the expected: True


[(1, 1), (2, 2), (3, 3)]

In [11]:
zipped_values = itertools.zip_longest(
    test_result,
    expected_result,
)

print(f"Test results match the expected: {all([i == j for i,j in zipped_values])}")

[*itertools.zip_longest(test_result, expected_result)]

Test results match the expected: False


[(1, 1), (2, 2), (3, 3), (None, 4), (None, 5)]

### A note about generators

`itertools` produces generators, so once we use them, they're empty e.g.

In [12]:
zipped_values = itertools.zip_longest(
    test_result,
    expected_result,
)

print([*zipped_values])
print([*zipped_values]) # this will be empty

[(1, 1), (2, 2), (3, 3), (None, 4), (None, 5)]
[]


### Another real world example

In [22]:
top_dict = {
"1": {"a": "a1", "b": "b1"},
"2": {"a": "a2", "b": "b2"},
"3": {"c": "a3", "b": "b3"},
}
a, b = [[v.get(k) for v in top_dict.values()] for k in "ab"]

In [23]:
a, b

(['a1', 'a2', None], ['b1', 'b2', 'b3'])

In [25]:
ab = [d.get(k) for k,d in itertools.product("ab", top_dict.values())]
a, b = ab[:len(top_dict)], ab[len(top_dict):] # wanted it in 2 lists
a, b

(['a1', 'a2', None], ['b1', 'b2', 'b3'])

In [ ]:
[*itertools.product("ab", [1,2])]

[('a', 1), ('a', 2), ('b', 1), ('b', 2)]

In [27]:
# The dark side of itertools!

a,b = itertools.zip_longest(
    *[
        (
            d.get(k) for k,d in itertools.product("ab", top_dict.values())
        )
    ]*len(top_dict)
)
a,b

(('a1', 'a2', None), ('b1', 'b2', 'b3'))

In [ ]:
gens = [(d.get(k) for k,d in itertools.product("ab", top_dict.values()))]*len(top_dict)
gens

[<generator object <genexpr> at 0x7f4eb578a8d0>,
 <generator object <genexpr> at 0x7f4eb578a8d0>,
 <generator object <genexpr> at 0x7f4eb578a8d0>]

In [ ]:
gens[0] is gens[1] is gens[2] # just points to the same generator!

True

In [ ]:
next(gens[0]), next(gens[1]), next(gens[2])

('a1', 'a2', None)

In [ ]:
# could actually just use zip here
[*zip(gens[0], gens[1], gens[2])]

[('b1', 'b2', 'b3')]

## Other examples

### Count

Can be used when the end point isn't known and can use non-integer steps:

In [14]:
import random

counter = itertools.count(start=0, step=0.5)

# Random length of array
[(x,y) for x,y in zip(counter, range(random.randint(0, 100)))]

[(0, 0),
 (0.5, 1),
 (1.0, 2),
 (1.5, 3),
 (2.0, 4),
 (2.5, 5),
 (3.0, 6),
 (3.5, 7),
 (4.0, 8),
 (4.5, 9),
 (5.0, 10),
 (5.5, 11),
 (6.0, 12),
 (6.5, 13),
 (7.0, 14),
 (7.5, 15),
 (8.0, 16),
 (8.5, 17),
 (9.0, 18),
 (9.5, 19),
 (10.0, 20),
 (10.5, 21),
 (11.0, 22),
 (11.5, 23),
 (12.0, 24),
 (12.5, 25),
 (13.0, 26),
 (13.5, 27),
 (14.0, 28),
 (14.5, 29),
 (15.0, 30),
 (15.5, 31),
 (16.0, 32),
 (16.5, 33),
 (17.0, 34),
 (17.5, 35),
 (18.0, 36),
 (18.5, 37),
 (19.0, 38),
 (19.5, 39),
 (20.0, 40),
 (20.5, 41),
 (21.0, 42),
 (21.5, 43),
 (22.0, 44),
 (22.5, 45),
 (23.0, 46),
 (23.5, 47)]

### Cycle

Cycles through an iterator, this could even be a generator! (see example below)

In [19]:
[(x,y) for x,y in zip(itertools.cycle(range(5)), range(20))]

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (0, 5),
 (1, 6),
 (2, 7),
 (3, 8),
 (4, 9),
 (0, 10),
 (1, 11),
 (2, 12),
 (3, 13),
 (4, 14),
 (0, 15),
 (1, 16),
 (2, 17),
 (3, 18),
 (4, 19)]

### Repeat

Repeats an object n times (n can be infinite)

In [23]:
[*itertools.repeat([0, 1], 3)]

[[0, 1], [0, 1], [0, 1]]

### accumulate

Keeps a running value, this could be the total, or the largest etc...

In [39]:
import operator

print([*itertools.accumulate(range(5))])
print([*itertools.accumulate(range(1, 6), operator.mul)])
print([*itertools.accumulate(range(1, 6), operator.gt)])
# Can use any custom functions
print([*itertools.accumulate(range(1, 6), max)])

[0, 1, 3, 6, 10]
[1, 2, 6, 24, 120]
[1, False, False, False, False]
[1, 2, 3, 4, 5]


In [46]:
[*itertools.accumulate(range(1, 6), lambda a,b: (a,b))]

[1, (1, 2), ((1, 2), 3), (((1, 2), 3), 4), ((((1, 2), 3), 4), 5)]

### compress

Takes in 2 iterables, the iterable to be compressed and an iterable of booleans of the same length, where the `True` values show which elements to keep.

In [ ]:
import string

compressed = itertools.compress(string.ascii_lowercase, [i%2 for i in range(len(string.ascii_lowercase))])

[*compressed] # select letters at odd indices

['b', 'd', 'f', 'h', 'j', 'l', 'n', 'p', 'r', 't', 'v', 'x', 'z']

### dropwhile

Drops elements of an iterable until a condition is false:

In [189]:
random_numbers = [random.randint(0, 10) for _ in range(20)]
random_numbers

[1, 10, 2, 0, 4, 8, 7, 6, 4, 10, 1, 2, 4, 4, 6, 6, 5, 5, 1, 2]

In [190]:
[*itertools.dropwhile(lambda x: x<9, random_numbers)]

[10, 2, 0, 4, 8, 7, 6, 4, 10, 1, 2, 4, 4, 6, 6, 5, 5, 1, 2]

In [193]:
# try it again with sorted values
[*itertools.dropwhile(lambda x: x<9, sorted(random_numbers))]

[10, 10]

### filterfalse

filters a list for when a condition is false:

In [200]:
[*itertools.filterfalse(lambda x: x<"m", string.ascii_lowercase)]

['m', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [205]:
# the opposite, we could just use this with a not, or in this case '>='
[*filter(lambda x: x<"m", string.ascii_lowercase)]

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']

### grouby

Groups elements of an iterable together by a key, the iterator must be sorted first though!

In [248]:
# creates an iterator that yields tuples like: ("a", 1) in random order
tuples = itertools.product("abc", [random.randint(0, 10) for _ in range(20)])

# sort the tuples by the random number!
sort_tuples = lambda x: x[1]

sorted_tuples = sorted(tuples, key=sort_tuples) # Not a generator
sorted_tuples

[('a', 0),
 ('a', 0),
 ('a', 0),
 ('b', 0),
 ('b', 0),
 ('b', 0),
 ('c', 0),
 ('c', 0),
 ('c', 0),
 ('a', 2),
 ('a', 2),
 ('b', 2),
 ('b', 2),
 ('c', 2),
 ('c', 2),
 ('a', 3),
 ('b', 3),
 ('c', 3),
 ('a', 5),
 ('a', 5),
 ('a', 5),
 ('a', 5),
 ('a', 5),
 ('a', 5),
 ('b', 5),
 ('b', 5),
 ('b', 5),
 ('b', 5),
 ('b', 5),
 ('b', 5),
 ('c', 5),
 ('c', 5),
 ('c', 5),
 ('c', 5),
 ('c', 5),
 ('c', 5),
 ('a', 6),
 ('a', 6),
 ('b', 6),
 ('b', 6),
 ('c', 6),
 ('c', 6),
 ('a', 7),
 ('a', 7),
 ('b', 7),
 ('b', 7),
 ('c', 7),
 ('c', 7),
 ('a', 8),
 ('b', 8),
 ('c', 8),
 ('a', 9),
 ('a', 9),
 ('b', 9),
 ('b', 9),
 ('c', 9),
 ('c', 9),
 ('a', 10),
 ('b', 10),
 ('c', 10)]

In [258]:
# returns a generator containing tuples of (key_value, generator_of_group)
groups = itertools.groupby(sorted_tuples, key=sort_tuples)

for kv, g in groups:
    print(kv, [*g])

0 [('a', 0), ('a', 0), ('a', 0), ('b', 0), ('b', 0), ('b', 0), ('c', 0), ('c', 0), ('c', 0)]
2 [('a', 2), ('a', 2), ('b', 2), ('b', 2), ('c', 2), ('c', 2)]
3 [('a', 3), ('b', 3), ('c', 3)]
5 [('a', 5), ('a', 5), ('a', 5), ('a', 5), ('a', 5), ('a', 5), ('b', 5), ('b', 5), ('b', 5), ('b', 5), ('b', 5), ('b', 5), ('c', 5), ('c', 5), ('c', 5), ('c', 5), ('c', 5), ('c', 5)]
6 [('a', 6), ('a', 6), ('b', 6), ('b', 6), ('c', 6), ('c', 6)]
7 [('a', 7), ('a', 7), ('b', 7), ('b', 7), ('c', 7), ('c', 7)]
8 [('a', 8), ('b', 8), ('c', 8)]
9 [('a', 9), ('a', 9), ('b', 9), ('b', 9), ('c', 9), ('c', 9)]
10 [('a', 10), ('b', 10), ('c', 10)]


In [269]:
# we could specify multiple sort keys e.g.

sort_tuples2 = lambda x: (x[1], x[0])
for kv, g in itertools.groupby(sorted(sorted_tuples, key=sort_tuples2), key=sort_tuples2):
    print(kv, [*g])

(0, 'a') [('a', 0), ('a', 0), ('a', 0)]
(0, 'b') [('b', 0), ('b', 0), ('b', 0)]
(0, 'c') [('c', 0), ('c', 0), ('c', 0)]
(2, 'a') [('a', 2), ('a', 2)]
(2, 'b') [('b', 2), ('b', 2)]
(2, 'c') [('c', 2), ('c', 2)]
(3, 'a') [('a', 3)]
(3, 'b') [('b', 3)]
(3, 'c') [('c', 3)]
(5, 'a') [('a', 5), ('a', 5), ('a', 5), ('a', 5), ('a', 5), ('a', 5)]
(5, 'b') [('b', 5), ('b', 5), ('b', 5), ('b', 5), ('b', 5), ('b', 5)]
(5, 'c') [('c', 5), ('c', 5), ('c', 5), ('c', 5), ('c', 5), ('c', 5)]
(6, 'a') [('a', 6), ('a', 6)]
(6, 'b') [('b', 6), ('b', 6)]
(6, 'c') [('c', 6), ('c', 6)]
(7, 'a') [('a', 7), ('a', 7)]
(7, 'b') [('b', 7), ('b', 7)]
(7, 'c') [('c', 7), ('c', 7)]
(8, 'a') [('a', 8)]
(8, 'b') [('b', 8)]
(8, 'c') [('c', 8)]
(9, 'a') [('a', 9), ('a', 9)]
(9, 'b') [('b', 9), ('b', 9)]
(9, 'c') [('c', 9), ('c', 9)]
(10, 'a') [('a', 10)]
(10, 'b') [('b', 10)]
(10, 'c') [('c', 10)]


### islice

Slices an iterable

In [280]:
[*itertools.islice(range(20), 4, 20, 2)]

[4, 6, 8, 10, 12, 14, 16, 18]

In [279]:
# can achieve the same thing using the `slice` keyword:
[*range(20)[slice(4, 20, 2)]]

[4, 6, 8, 10, 12, 14, 16, 18]

In [283]:
# slice also supports negative indices, whereas islice does not
[*range(20)[slice(-4, 20, 2)]]

[16, 18]

In [312]:
# So why use islice?
list_vals = [*range(20)]
print(list_vals)

# slice returns a new list, islice returns a generator!
print(list_vals[slice(5, 20, 5)])
itertools.islice(list_vals, 5, 20, 5)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[5, 10, 15]


### starmap

maps a function to an iterable, used when the values are alreday grouped together into e.g. tuples

In [324]:
x = [*range(10)]
vals = [(x[i], x[i+1], x[i+2]) for i in range(len(x)-2)]
vals += [(8, 9)]
vals

[(0, 1, 2),
 (1, 2, 3),
 (2, 3, 4),
 (3, 4, 5),
 (4, 5, 6),
 (5, 6, 7),
 (6, 7, 8),
 (7, 8, 9),
 (8, 9)]

In [325]:
def starmap_test(*args):
    if len(args) ==3:
        return (args[0]+args[1])*args[2]
    else:
        return args[0]*args[1]

In [326]:
[*itertools.starmap(starmap_test, vals)]

[2, 9, 20, 35, 54, 77, 104, 135, 72]

In [335]:
# map doesn't unpack, so it sees a tuple of a tuple given to it
[*map(starmap_test, vals)]

IndexError: tuple index out of range

In [339]:
def map_test(*args):
    print(args, len(args))

for x in map(map_test, vals):
    pass

((0, 1, 2),) 1
((1, 2, 3),) 1
((2, 3, 4),) 1
((3, 4, 5),) 1
((4, 5, 6),) 1
((5, 6, 7),) 1
((6, 7, 8),) 1
((7, 8, 9),) 1
((8, 9),) 1


In [341]:
# We can achieve the same thing here with map by not unpacking the tuple:
def map_test2(args):
    if len(args) ==3:
        return (args[0]+args[1])*args[2]
    else:
        return args[0]*args[1]
    
[*map(map_test2, vals)]

[2, 9, 20, 35, 54, 77, 104, 135, 72]

### takewhile

Roughly the opposite of `itertools.dropwhile`

In [381]:
random_numbers = [random.randint(0, 10) for _ in range(20)]
random_numbers

[8, 5, 6, 2, 10, 8, 5, 8, 8, 9, 8, 1, 8, 3, 7, 6, 2, 1, 1, 5]

In [383]:
[*itertools.takewhile(lambda x: x<9, random_numbers)], [*itertools.dropwhile(lambda x: x<9, random_numbers)]

([8, 5, 6, 2], [10, 8, 5, 8, 8, 9, 8, 1, 8, 3, 7, 6, 2, 1, 1, 5])

### tee

As we've seen throughout this notebook, once a generator is used, it returns an empty object.

`tee` allows us to 'copy' the generator for multiple use!

In [373]:
vals = zip(range(10), string.ascii_lowercase)
print([*vals])
[*vals]

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f'), (6, 'g'), (7, 'h'), (8, 'i'), (9, 'j')]


[]

In [378]:
vals = zip(range(10), string.ascii_lowercase)

dup_vals = itertools.tee(vals, 2)
dup_vals

(<itertools._tee at 0x7f4eb4cfae10>, <itertools._tee at 0x7f4eb4cfaf50>)

In [379]:
print(next(dup_vals[0])) # lets remove the first element of the first generator

print([*dup_vals[0]])
print([*dup_vals[1]])

(0, 'a')
[(1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f'), (6, 'g'), (7, 'h'), (8, 'i'), (9, 'j')]
[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f'), (6, 'g'), (7, 'h'), (8, 'i'), (9, 'j')]


Note: There are a few caveats with tee, it's best to read through the documentation on https://docs.python.org/3.7/library/itertools.html#itertools.tee

### permutations

Returns all the permutations of an iterable with the length `r`:

In [391]:
[*itertools.permutations("abc", r=3)]

[('a', 'b', 'c'),
 ('a', 'c', 'b'),
 ('b', 'a', 'c'),
 ('b', 'c', 'a'),
 ('c', 'a', 'b'),
 ('c', 'b', 'a')]

### combinations

Finds all the combinations of an iterable with length `r`, where no element is repeated!

In [401]:
[*itertools.combinations("abc", r=3)]

[('a', 'b', 'c')]

### combinations_with_replacement

Finds all the combinations of an iterable with length `r`, where every element can be repeated:

In [402]:
[*itertools.combinations_with_replacement("abc", r=3)]

[('a', 'a', 'a'),
 ('a', 'a', 'b'),
 ('a', 'a', 'c'),
 ('a', 'b', 'b'),
 ('a', 'b', 'c'),
 ('a', 'c', 'c'),
 ('b', 'b', 'b'),
 ('b', 'b', 'c'),
 ('b', 'c', 'c'),
 ('c', 'c', 'c')]

## Conclusion

`itertools` can be really powerful, especially when combined together. But remember, code readability is important! If you find that a nested `for` loop is more readable, then use it! Hopefully seeing these examples gives you more options to use the correct tool for the job.

For some practice using `itertools`, see https://www.w3resource.com/python-exercises/itertools/index.php